# Music Information Retrieval

## Secuencias de acordes

### Algoritmo de deteccion de acordes en una cancion

La deteccion de acordes en APICultor tiene como objetivo obtener los acordes en una cancion que sigan una tonalidad. Para obtener las notas musicales, escalas que siguen y la relacion de la nota musical detectada con la segunda maxima correlacion utilizamos perfiles de acordes de los cuales se reconstruyen perfiles de clase tonal para comparar con el perfil de PCP de entrada utilizado para saber cual es la nota y escala que sigue ese perfil (el cual tiene que ser el acorde hallado en esa entrada de la cancion).
Luego de obtener correlaciones de todas las entradas la intencion en nuestra modificacion del algoritmo original es purgar la salida de acordes detectados que no respetan la tonalidad de toda la cancion. Para hacer esto se puede afirmar mediante analisis que los acordes detectados que se corresponden con los de una cancion analizada mantienen una relacion muy fuerte con otras correlaciones, por lo que mientras un acorde detectado aparece con mas fuerza que los demas en realidad esta bastante alejado de la tonalidad que sigue toda la cancion.

#### Deteccion de acordes en APICultor

##### Calculando onsets

El primer paso antes de saber cuales son los acordes de una cancion lo recomendable es utilizar solo las secciones de una cancion donde hayan onsets basados en la complejidad que se puede obtener de un sonograma. Dada una fase queremos saber la distancia respecto de las que pueden haber en todo el espectro sabien asi donde pueden llegar a haber harmonicos 

In [41]:
#importamos las funciones en el modulo de algoritmos MIR
from apicultor.utils.algorithms import * 

#importamos soundfile de las dependencias de apicultor
from soundfile import read 

tags_dir = 'bajo' 

#abrimos nuestro sonido de RedPanal
audio, fs = read('710.ogg') 

#lo convertimos a mono para poder analizar
audio = mono_stereo(audio) 

#llamamos a la clase MIR para obtener nuestros onsets
retrieve = MIR(audio, fs) 

#aplicamos un filtro para remover el ruido hasta 40 Hz
retrieve.signal = retrieve.IIR(retrieve.signal, 40, 'highpass')

retrieve.audio_signal_spectrum = []                                                                                                          
for i in retrieve.FrameGenerator():                                                           
        retrieve.window()       #ventaneamos el cuadro que analizamos                                             
        retrieve.Spectrum()     #computamos la magnitud espectral del cuadro ventaneado                                                                                                  
        retrieve.Phase(retrieve.fft(retrieve.windowed_x)) #computamos la fase de la transformada de fourier de la onda ventaneada         
        onsets.append(retrieve.detect_by_polar()) #hallamos la distancia polar entre las fases y las magnitudes de un espectro
        
retrieve.onsets_by_polar_distance(onsets) #los onsets deberian ser los onsets con mayor distanciasets

##### Computando los perfiles harmonicos de clase tonal

Los perfiles harmonicos de clase tonal nos permiten visualizar la contribucion de los harmonicos en un espectrograma para tener una intuicion sobre las notas musicales que aparecen en el espectrograma mismo. Se utilizan solamente las frecuencias y magnitudes que se supone que estan dentro de un rango de parciales de tonos musicales para computar los valores del perfil. Las contribuciones harmonicas se basan simplemente en la distancia entre los bines del perfil de clase tonal manteniendo la compresion logaritmica que hacemos sobre los harmonicos.
El numero de bins que utilizamos es 12, al igual que el numero de harmonicos para que la contribucion represente el peso que tiene cada semitono en un espectrograma planteando el hecho de que cada harmonico debe corresponderse con un bin de frecuencia el cual debera unirse con el de una nota musical deducible a partir del espectro.

Una vez obtenidos nuestros onsets computaremos el perfil de clase tonal de todos los espectrogramas.

In [45]:
hpcps = []
for i in retrieve.audio_signal_spectrum:
    retrieve.magnitude_spectrum = i
    retrieve.spectral_peaks()
    hpcps.append(hpcp(retrieve,12))
hpcps = np.array(hpcps)

[array([0.18658128, 0.0190841 , 0.28505922, 0.04621067, 1.93077631,
       0.12821905, 0.07392931, 0.14479948, 0.00211425, 1.07146008,
       0.00982275, 1.00673982]), array([0.56847433, 0.61374171, 0.01133067, 0.25798968, 0.02085195,
       1.11492752, 0.2983381 , 1.0961489 , 0.00955779, 0.04326692,
       0.54399257, 0.04833057]), array([0.18015475, 0.19962644, 0.49059591, 0.03395782, 0.98858593,
       0.08877685, 0.11072783, 0.33542746, 0.52253326, 1.13591667,
       0.09897926, 1.04263226]), array([1.5701587 , 0.11806992, 0.94319018, 0.58734602, 0.14922758,
       0.59353641, 0.75850534, 1.00879555, 0.97515878, 0.12737603,
       1.1365411 , 0.20465039]), array([0.80442703, 1.23271115, 0.80317463, 0.41577246, 1.17760946,
       0.42361212, 0.94336537, 0.27800689, 1.64134408, 1.        ,
       1.00170537, 0.12599385]), array([1.02839164, 0.84690703, 0.58215993, 0.17496854, 0.5331389 ,
       0.84457946, 0.55152797, 0.79674938, 1.80907098, 0.01882169,
       0.51736524, 0.07679495]

##### Detectar acordes en la pieza musical

La deteccion de acordes no difiere mucho de los metodos utilizados corrientemente. Utilizando varios perfiles harmonicos buscamos mediante correlacion cruzada cual es el perfil con mayor valor de correlacion para determinar la escala y la nota detectada a partir del perfil harmonico de clase tonal.
Los perfiles harmonicos con los que se realiza el analisis de correlacion son perfiles reconstruidos a partir de pistas MIDI que reproducen los acordes, los mismos se pueden encontrar haciendo una busqueda en Wikipedia.
El desafio que presenta el analisis de acordes es el de encontrar correlaciones que demuestren que los perfiles utilizados corresponden con los acordes de la cancion y asi utilizarlos para detectar la secuencia tonal de la cancion. Respecto a esto podemos afirmar que cuanto mas fuerte es la relacion entre las correlaciones maximas de cada analisis es posible alcanzar la nota musical que aparece en el perfil harmonico. El problema (aun pendiente) por resolver es por un lado entender el efecto de asociar una nota musical a la secuencia tonal respecto de las demas notas detectadas ya que una asociacion muy fuerte podria indicar que la asociacion correcta en realidad deberia estar en otra parte, lo cual significa mejorar el analisis de correlacion cruzada, modificar parametros en la computacion de los perfiles harmonicos u optimizar aun mas el analisis.
Se puede demostrar con nuestro analisis que si la fuerza de un valor maximo respecto al proximo valor maximo es abrupta queda en evidencia la ausencia de simetria necesaria para configurar una tonalidad musical, por ende esa asociacion no deberia considerarse para estimar la secuencia de acordes.
De ahi, podria decirse que aquellos valores cercanos a la secante y cotangente hiperbolicas de todos los acordes serian los que mantienen una estructura tonal especifica que se correlaciona con los tonos presentes en la cancion detectada. Esto no hace mas que resaltar la necesidad de obtener correlaciones mas aproximadas a la hora de entender la tonalidad de la cancion.

##### Detectando acordes en APICultor

Detectar acordes es muy sencillo, simplemente llamaremos a la funcion chord_sequence utilizando los perfiles harmonicos de clase tonal como argumento de la funcion.

In [66]:
chords = chord_sequence(hpcps)
